In [3]:
from fastai.vision.all import *
from fastai.data.transforms import get_image_files
import torchvision.transforms as transforms

In [5]:
import torch
import numpy as np
import PIL

In [6]:
def open_image(fname, size=224):
    img = PIL.Image.open(fname).convert('RGB')
    img = img.resize((size, size))
    t = torch.Tensor(np.array(img))
    return t.permute(2,0,1).float()/255.0

In [8]:
# image transform 
class ImageTransform():
    def __init__(self, mean, std):
        self.data_transform = transforms.Compose([
            transforms.ToTensor(),
            transforms.Normalize(mean, std)
        ])
        
    def __call__(self, img):
        return self.data_transform(img)

class ImageDataset(torch.utils.data.Dataset):
    def __init__(self,files, labels):
        self.files, self.labels = files, labels
    
    def __getitem__(self, i):
        file = self.files[i]
        
        img = open_image(file)
        label = self.labels[i]

        return (img, label)

    def __len__(self): return len(self.files)

In [9]:
def make_file_list(path):
    train_img_list = list()

    with os.scandir(path) as files:
        for file in files:
            if file.name.endswith('.jpg'):
              # adds only the image files to the flowers list
                train_img_list.append(f"{path}/{file.name}")
    
    return train_img_list

In [ ]:
# make train dataloader list

lie_img_path = r"./dataset/lie"
not_lie_img_path = r"./dataset/not_lie"

lie_files = make_file_list(lie_img_path)
not_lie_files = make_file_list(not_lie_img_path)

files = lie_files + not_lie_files
labels = ["lie" ]*len(lie_files)+["not_lie"]*len(not_lie_files)

not_lie_files[0]

In [ ]:
idxs = np.random.permutation(range(len(files)))
cut = int(0.8 * len(files))
train_files = files[idxs[:cut]]
valid_files = files[idxs[cut:]]

In [ ]:
train_ds = ImageDataset(train_files)
valid_ds = ImageDataset(valid_files)

In [ ]:
from fastai.data.core import DataLoaders

dls = DataLoaders.from_dsets(train_ds, valid_ds)

In [ ]:
learn = cnn_learner(dls, resnet34, metrics=error_rate, pretrained=True)
learn.fine_tune(epochs=1)

In [ ]:
img = open_image("file_name")

label, _, probs = learn.predict(img)


print(f"You are {label}")
print(probs)

In [ ]:
learn.export()

In [ ]:
# https://colab.research.google.com/github/Sebastian-Schuchmann/ChurrorsSamosaClassifier/blob/main/Train_a_Food_Model.ipynb#scrollTo=KddD9EmLdon0

#let's grab the first pkl file we can find
modelPath = get_files(foodPath, '.pkl')[0]
modelPath

learn_inf = load_learner(modelPath)
learn_inf.predict(mpimg.imread(get_image_files(foodPath)[0])) #raw prediction

learn_inf.dls.vocab #Get the labels